In [1]:
#Importation
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sklearn as sk
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from statistics import mean
from math import ceil
from collections import Counter
warnings.filterwarnings('ignore')

In [2]:
# On ramene les données du prétraitement du dataframe
df= pd.read_pickle("../PICKLE/df_stat_commentaires.pkl")
df

,score_reussite,score_echec,nombre_commentaires,Longueur moyenne commentaire,Reussite,polarite,subjectivite
0,-1.365733,-0.869921,-0.755824,-0.328797,False,0.000000,0.000000
1,0.457547,0.164916,0.333062,-0.468411,True,0.100000,0.100000
2,1.567866,1.617878,1.966390,1.752941,True,0.062500,0.062500
3,-0.173371,-0.744961,-0.755824,-0.627969,True,0.133333,0.158333
4,-0.543328,-0.751616,-0.755824,-0.877279,True,0.166667,0.166667
...,...,...,...,...,...,...,...
4183,-0.191482,0.146861,1.966390,-0.359961,True,0.000000,0.000000
4184,0.065179,-0.596722,-0.755824,-0.665366,True,0.166667,0.166667
4185,-0.582355,-0.757317,-0.755824,-0.627969,True,0.166667,0.166667
4186,-0.305364,-0.483163,0.333062,-0.722707,True,0.100000,0.100000


In [3]:
def creation_test_train(dataframe,label):
    train_df = dataframe.sample(frac=0.8)
    test_df = dataframe.drop(train_df.index)

    train_df_Y = train_df[label]
    test_df_Y = test_df[label]

    train_df_X = train_df.drop(label,axis=1)
    test_df_X = test_df.drop(label,axis=1)
    
    return train_df_X,train_df_Y,test_df_X,test_df_Y

In [4]:
def stat_modele(true,pred,arrondi=5,affichage=True):
    accuracy = round(sk.metrics.accuracy_score(true,pred),arrondi)*100
    rappel = round(sk.metrics.recall_score(true,pred),arrondi)*100
    precision = round(sk.metrics.precision_score(true,pred),arrondi)*100
    specificity = round(sk.metrics.recall_score(true,pred,pos_label=0),arrondi)*100
    vpn = round(sk.metrics.precision_score(true,pred,pos_label=0),arrondi)*100
    if affichage :
        print("Accuracy : proportion de prédictions correctes (VP + VN /(VP+VN+FN+FP)) : " + str(accuracy) + "%")
        print("Rappel : proportion de réussites correctement prédites sur l'ensemble des réussites (VP/(VP+FN)) : " + str(rappel) + "%")
        print("Précision :  proportion de véritables réussites parmi les réussites prédites (VP/(VP+FP)) : " + str(precision) + "%")
        print("Spécificité : proportion d'échecs correctement prédits sur l'ensemble des échecs (VN/(VN+FP)) : "+str(specificity) + "%")
        print("Valeur prédictive négative: proportion de véritables échecs parmi les échecs prédits (VN/(VN+FN)) : " + str(vpn) + "%")
   
    return accuracy,rappel,precision,specificity,vpn

In [5]:
def voisinage():
    train_df_X,train_df_Y,test_df_X,test_df_Y = creation_test_train(df,"Reussite")
    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(train_df_X, train_df_Y)
    stat_modele(test_df_Y,neigh.predict(test_df_X),arrondi=5,affichage=True)
    return neigh

### Premier modèle

In [6]:
voisinage()

Accuracy : proportion de prédictions correctes (VP + VN /(VP+VN+FN+FP)) : 93.31700000000001%
Rappel : proportion de réussites correctement prédites sur l'ensemble des réussites (VP/(VP+FN)) : 96.519%
Précision :  proportion de véritables réussites parmi les réussites prédites (VP/(VP+FP)) : 96.005%
Spécificité : proportion d'échecs correctement prédits sur l'ensemble des échecs (VN/(VN+FP)) : 67.033%
Valeur prédictive négative: proportion de véritables échecs parmi les échecs prédits (VN/(VN+FN)) : 70.11500000000001%


KNeighborsClassifier()

### Modèle avec le seuil

In [45]:
def arbre_seuil():
    train_df_X,train_df_Y,test_df_X,test_df_Y = creation_test_train(df,"Reussite")
    neigh = KNeighborsClassifier(n_neighbors=5,weights="uniform",p=1)
    neigh.fit(train_df_X, train_df_Y)
    predictions = neigh.predict_proba(test_df_X)
    seuil = 0.9
    predict_label_seuil = [False if proba[0]>=seuil else True for proba in predictions]
    stat_modele(test_df_Y,predict_label_seuil,arrondi=5,affichage=True)
    return neigh

In [54]:
arbre_seuil()

Accuracy : proportion de prédictions correctes (VP + VN /(VP+VN+FN+FP)) : 92.243%
Rappel : proportion de réussites correctement prédites sur l'ensemble des réussites (VP/(VP+FN)) : 99.864%
Précision :  proportion de véritables réussites parmi les réussites prédites (VP/(VP+FP)) : 92.01%
Spécificité : proportion d'échecs correctement prédits sur l'ensemble des échecs (VN/(VN+FP)) : 36.0%
Valeur prédictive négative: proportion de véritables échecs parmi les échecs prédits (VN/(VN+FN)) : 97.297%


KNeighborsClassifier(p=1)

### Amélioration des hyper-paramètres

In [29]:
def BestHyperparameters_vpn(neighbors,weights,p):
    best_vpn = 0
    best_n_neighbor = 0
    best_weight = 0
    best_p = 0
    trainX,trainY,testX,testY = creation_test_train(df,"Reussite")
    
    #On teste toutes les combinaisons possibles.
    
    for n_neighbor in neighbors :
        for weight in weights :
            for i in p :
                #On crée le voisinage
                neigh = KNeighborsClassifier(n_neighbors=n_neighbor,weights=weight,p=i)
                neigh.fit(trainX, trainY)
                # On calcule la VPN associée
                predictionY  = neigh.predict(testX)
                vpn = round(sk.metrics.precision_score(testY,predictionY,pos_label=0),5)*100
                #On compare
                if vpn>best_vpn :
                    best_vpn=vpn
                    best_n_neighbor = n_neighbor
                    best_weight = weight
                    best_p = i
    return (best_vpn,best_n_neighbor,best_weight,best_p)

In [30]:
BestHyperparameters_vpn([3,5,7],["uniform","distance"],[1,2])

(82.55799999999999, 5, 'distance', 2)

In [28]:
def BestHyperparameters_vpn_seuil(neighbors,weights,p):
    best_vpn = 0
    best_n_neighbor = 0
    best_weight = 0
    best_p = 0
    trainX,trainY,testX,testY = creation_test_train(df,"Reussite")
    
    #On teste toutes les combinaisons possibles.
    
    for n_neighbor in neighbors :
        for weight in weights :
            for i in p :
                #On crée le voisinage
                neigh = KNeighborsClassifier(n_neighbors=n_neighbor,weights=weight,p=i)
                neigh.fit(trainX, trainY)
                # On calcule la VPN associée
                predictions = neigh.predict_proba(testX)
                seuil = 0.9
                predictionY  = [False if proba[0]>=seuil else True for proba in predictions]
                vpn = round(sk.metrics.precision_score(testY,predictionY,pos_label=0),5)*100
                #On compare
                if vpn>best_vpn :
                    best_vpn=vpn
                    best_n_neighbor = n_neighbor
                    best_weight = weight
                    best_p = i
    return (best_vpn,best_n_neighbor,best_weight,best_p)

In [34]:
BestHyperparameters_vpn_seuil([3,5,7],["uniform","distance"],[1,2])

(95.652, 7, 'uniform', 1)

In [41]:
def Finetune_vpn_seuil(essais,neighbors,weights,p):
    l_vpn = []
    l_best_n_neighbor = []
    l_best_weight = []
    l_best_p = []
    
    for i in range (0,essais+1):
        if i%100 == 0:
            print(i)
        best_vpn,best_n_neighbor,best_weight,best_p = BestHyperparameters_vpn_seuil(neighbors,weights,p)
        l_vpn.append(best_vpn)
        l_best_n_neighbor.append(best_n_neighbor)
        l_best_weight.append(best_weight)
        l_best_p.append(best_p)
    return mean(l_vpn), Counter(l_best_n_neighbor).most_common(1)[0][0],Counter(l_best_weight).most_common(1)[0][0],Counter(l_best_p).most_common(1)[0][0]

In [42]:
Finetune_vpn_seuil(1000,[3,5,7],["uniform","distance"],[1,2])

0
100
200
300
400
500
600
700
800
900
1000


(97.2691958041958, 5, 'uniform', 1)